In [3]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

If you worked collaboratively on this assignment, __include the names of your collaborators in the cell below:__



## 8.0 - Homework correction (3 points)

In the cell below, explain and correct __one mistake__ you made on homework #6.  If you got a perfect score, great, tell us what your favorite problem was, or use the space to give any other feedback you might have on the class/tutorials/homework.

### Important Note

Once again, this is a "mini-project" homework assignment: __there is only one problem.__  Again, this is a bit of a warm-up for the final projects.

To give you a bit of extra time to put the pieces together, __you will have 2 weeks to complete this assignment instead of the usual 1 week.__

## 8.1 - Hitting the mark with air resistance (45 points)

_(Note: __this is the only problem on this week's homework!__  Think of this as a miniature version of the projects that you'll be doing at the end of the semester.  You should start early to make sure you have time to chat with the instructors and other students if you get stuck...)_

We've done a number of kinematics problems, but most of them didn't include anything you didn't already know the answer to.  Now that we can solve ODEs, we can go far beyond what pen and paper can do!

Let's consider __two-dimensional kinematics with air resistance.__ For relatively small speed (and/or small objects), the effect of air resistance is to give a _drag_ force which is proportional to the speed:

$$
\mathbf{F}_{\rm drag} = -b\mathbf{v}
$$

For larger objects and higher speeds, a _quadratic_ drag force becomes important,

$$
\mathbf{F}_{\rm drag, quad} = - cv\mathbf{v}
$$

(where $c$ itself depends on some things like [air density and the size of our object](http://hyperphysics.phy-astr.gsu.edu/hbase/airfri.html), but we'll just assume those are all fixed.)  The units of $b$ are kg/s, and for $c$ we have kg/m.

If we add in gravity as well, the two-dimensional kinematic equations are, split into x- and y-components,

$$
m \frac{d^2x}{dt^2} = -bv_x -  c v_x \sqrt{v_x^2 + v_y^2} \\
m \frac{d^2y}{dt^2} = -mg - bv_y -  c v_y \sqrt{v_x^2 + v_y^2}
$$

### Part A (6 points)

To solve this as an initial-value problem using our ODE methods, we need to split this into _four first-order ODEs_ for the quantities $x$, $v_x$, $y$ and $v_y$.  That's easy enough to do:

$$
\frac{dx}{dt} = v_x \\
\frac{dy}{dt} = v_y \\
\frac{dv_x}{dt} = -bv_x - cv_x \sqrt{v_x^2 + v_y^2} \\
\frac{dv_y}{dt} = -mg - bv_y - cv_y \sqrt{v_x^2 + v_y^2} 
$$

Following the procedure described in lecture, we now convert to finite-difference equations by using the first-order forward discrete derivative assuming we've made everything into a discrete set of samples $t_i, x_i, y_i...$, with a fixed grid spacing $\Delta t$ between our points in time.

For example, we replace

$$
\frac{dx}{dt} \rightarrow \frac{x_{i+1} - x_i}{\Delta t}
$$

which converts the first equation above into

$$
x_{i+1} = x_i + v_{x,i} \Delta t 
$$

__In the Markdown cell below, write out the other three finite-difference equations using TeX math:__

$$
y_{i+1} = ? \\
v_{x,i+1} = ? \\
v_{y,i+1} = ?
$$

(Remember, you can double-click on this cell to copy the Markdown source for those equations; then fill out the right-hand sides.  Notice the `\\`, which in TeX notation at the end of a line means "start a new equation line after this".)

Discretizing the $dy/dt$ equation gives basically the same equation as the one for $x_{i+1}$:

$$
y_{i+1} = y_i + v_{y,i} \Delta t
$$

The other two equations are more complicated, but it's not any harder to discretize!  For example, once we convert

$$
\frac{dv_x}{dt} \rightarrow \frac{v_{x,i+1} - v_{x,i}}{\Delta t}
$$

then we just reorganize the equation to get

$$
v_{x,i+1} = v_{x,i} - \Delta t (b v_{x,i} + cv_{x,i} \sqrt{v_{x,i}^2 + v_{y,i}^2})
$$

and similarly,

$$
v_{y,i+1} = v_{y,i} - \Delta t (mg + b v_{y,i} + cv_{y,i} \sqrt{v_{x,i}^2 + v_{y,i}^2})
$$

### Part B (14 points)

Now that we have equations, we can use __Euler's method__ to solve them, as we did in the tutorials:

* Evaluate the right-hand side of the difference equations at step $i$;
* Store the results as the value of each variable at step $i+1$;
* Repeat for the next step, until we reach the end of the trajectory.

For simplicity, we'll fix $g = 9.8\ m/s^2$ and $m = 1$ kg, and the motion will always start from $(x_0, y_0) = (0,0)$.



__Implement the function `solve_motion(t, b, c, v_0)` below__, which should solve the kinematic equations you wrote down in part A taking the following inputs:

* `t`, a discrete list of time values over which we'll be solving the equation.  (For example, if we want to solve the motion for 5 seconds using 200 steps, we would call `solve_motion` with `t=np.linspace(0,5,200)`.)
* `b` and `c`, the drag coefficients appearing above;
*  `v_0` is a length-two tuple containing the initial conditions for speed in both x and y, in the form `v_0 = (v_x0, v_y0)`.

The function __should return two arrays `x` and `y`__ which contain the results for the trajectory at each timestep, $x_i$ and $y_i$.

I didn't give you a docstring or comments to guide you this time; __maybe you should write your own before you start!__ (If you get stuck, you should _definitely_ write out what you're trying to do in comments and docstring.)

(__Hint:__ if you get stuck planning out the function `solve_motion`, try to follow what we did on tutorial 15: write a separate function `compute_next_step(x, y, vx, vy, dt)` that returns `(next_x, next_y, next_vx, next_vy)`, and then incorporate that into your big function `solve_motion`.)



In [4]:
def solve_motion(t, v_0, b, c):
    # Add docstring here!
    
    g = 9.8  # m/s^2
    m = 1    # kg
    
    ### BEGIN SOLUTION
    # Find dt and N
    dt = t[1] - t[0]
    Nt = len(t)
    
    # Initialize arrays for our trajectory
    x = np.zeros_like(t)
    y = np.zeros_like(t)
    vx = np.zeros_like(t)
    vy = np.zeros_like(t)
    
    # Set initial values
    vx[0], vy[0] = v_0
    
    # Iterate and apply the Euler step
    for i in range(0, Nt - 1):
        x[i+1] = x[i] + vx[i] * dt
        y[i+1] = y[i] + vy[i] * dt
        
        speed_i = np.sqrt(vx[i]**2 + vy[i]**2)
        
        vx[i+1] = vx[i] + dt * (-b*vx[i] - c*vx[i] * speed_i)
        vy[i+1] = vy[i] + dt * (-m*g - b*vy[i] - c*vy[i] * speed_i)
        
    return x,y
    
    
    ### END SOLUTION

There are a couple of simple tests that you can run to make sure things are working correctly in your solver above:

* If you set both `b` and `c` to zero, you should get regular motion with no drag.  In particular, remember that the range of a projectile fired from the origin with no drag is:

$$
R = \frac{v_0^2 \sin (2\theta)}{g} 
$$

* If `c` is zero but `b` is not, then the motion [can be solved analytically](http://farside.ph.utexas.edu/teaching/336k/Newtonhtml/node29.html) _(be aware: the link uses different notation than we do!)_.  In particular, in the limit that `b` is very large, the range becomes

$$
R = \frac{m v_0 \cos \theta}{b}
$$

If we set the initial speed to $(v_x, v_y) = (4,4)$ m/s, then $v_0 = 4\sqrt{2} = 5.657$ m/s.  Plugging in numbers, we can find

$$
R = 3.265\ \rm{m}\ \ \textrm{(no drag)} $$

$$
R = 0.133\ \rm{m}\ \ (\textrm{extreme drag}, b = 30\ \rm{kg/s})
$$

Make sure your numerical solution gets both of these right!  I've given you a range-finding function below that takes x,y trajectories as `solve_motion` should return them.



In [5]:
# Range-finder for trajectories (x(t), y(t)).
# Note that this will have a numerical error depending on dt.
# If your answer looks wrong, first try adding more steps!

def find_range(x,y):
    
    # Cut off the first few entries, since y starts
    # at exactly zero and we don't want to find x=0!
    i_zero = np.abs(y[3:]).argmin() 
    i_zero += 3  # i_zero is off by 3 since we cut off y; fix it!

    return x[i_zero]

In the cell below, check your code by __calculating the motion for `b=0`, `c=0`__ over a time range of 5 seconds.  __Plot the trajectory__ and make sure it looks reasonable.  Then in the next cell, __use `find_range` to check the range__ against the formula above.

(You may write an automatic test with `assert` or `numpy.testing.assert_allclose`, or you can check the range by hand.  But show your work either way!)

In [6]:
# Check/plot your solver for b=0, c=0 here

### BEGIN SOLUTION
t = np.linspace(0,5,50000)
x,y = solve_motion(t, (4, 4), b=0, c=0.)

plt.plot(x,y)
plt.xlim(0,3.5)
plt.ylim(0,1)
### END SOLUTION

(0.0, 1.0)

In [7]:
# Test the range against the analytic result here!
known_R = 3.2653

### BEGIN SOLUTION
print("Range =", find_range(x,y))
### END SOLUTION

Range = 3.2656653133065494


Now, air resistance!  In the cell below, check your code by __calculating the motion for `b=30` kg/s, `c=0`__ over a time range of 5 seconds.  __Plot the trajectory__ and make sure it looks reasonable.  Then in the next cell, __use `find_range` to check the range__ against the formula above.

(You may write an automatic test with `assert` or `numpy.testing.assert_allclose`, or you can check the range by hand.  But show your work either way!)

In [8]:
# Check/plot your solution for b=30 kg/s, c=0  here

### BEGIN SOLUTION
x,y = solve_motion(t, (4,4), b=30, c=0)
print("Range =", find_range(x,y))

plt.plot(x,y)
plt.ylim(0,0.12)
### END SOLUTION

Range = 0.13333310215491428


(0.0, 0.12)

In [9]:
# Hidden testing cell; do not delete!
import numpy.testing as npt

### BEGIN HIDDEN TESTS
t = np.linspace(0,5,20000)
x1,y1 = solve_motion(t, (4,4), b=0, c=0)
x2,y2 = solve_motion(t, (4,4), b=30, c=0)

print("x1[0:10] = ", x1[0:10])
print("y1[0:10] = ", y1[0:10])
print("x2[0:10] = ", x2[0:10])
print("y2[0:10] = ", y2[0:10])

npt.assert_allclose(x1[y1>=0][-1], 3.2653, atol=1e-3)
npt.assert_allclose(x2[y2>=0][-1], 0.1333, atol=1e-3)

npt.assert_allclose(y1[1000], 0.6941, atol=1e-3)
npt.assert_allclose(y2[1000], 0.0625, atol=1e-3)
### END HIDDEN TESTS

x1[0:10] =  [0.         0.00100005 0.0020001  0.00300015 0.0040002  0.00500025
 0.0060003  0.00700035 0.0080004  0.00900045]
y1[0:10] =  [0.         0.00100005 0.00199949 0.00299831 0.00399652 0.00499412
 0.00599111 0.00698749 0.00798325 0.0089784 ]
x2[0:10] =  [0.         0.00100005 0.0019926  0.0029777  0.00395542 0.0049258
 0.00588891 0.00684479 0.0077935  0.0087351 ]
y2[0:10] =  [0.         0.00100005 0.00199199 0.00297587 0.00395176 0.00491972
 0.00587981 0.00683208 0.0077766  0.00871343]


##  Part C (8 points): an alternate solution using Runge-Kutta

Another way to approach this problem is to use the Runge-Kutta, the fourth-order method mentioned in tutorial 15. To do this, we need to define a function that computes the first-order derivatives, then pass that function to the Runge-Kutta solver.

When setting up a system of ODEs, we need to construct an array of the derivatives and a function to make that array. Going back to our system, we can write them as a first-order system of these equations:
$$
\frac{dx}{dt} = v_x \\
\frac{dy}{dt} = v_y \\
\frac{dv_x}{dt} = -bv_x - cv_x \sqrt{v_x^2 + v_y^2} \\
\frac{dv_y}{dt} = -mg - bv_y - cv_y \sqrt{v_x^2 + v_y^2} 
$$
Your next goal is to write a function that will calculate these. Therefore, you want a function that will return `y`, where `y[0] ` corresponds to $\frac{dx}{dt}$, `y[1]` corresponds to $\frac{dy}{dt}$, `y[2]` corresponds to $\frac{dv_x}{dt} $, and `y[3]` corresponds to $\frac{dv_y}{dt} $, computed from the right-hand sides of the equations above.

Your function will also need to know the values of the parameters `m, g, b, c`, so pass those as arguments.

In the cell below, write the function `ode_RHS` that calculates the derivatives and returns the array `y`. Then define initial values for your function `x0, y0, vx0, vy0` to be passed to `si.solve_ivp`. 



In [10]:
import scipy.integrate as si


# define function to pass to scipy.integrate, then define the constants you will need for the solution
def ode_RHS(t, y, m=1, g=9.8, b=0, c=0):
    ### BEGIN SOLUTION
    x, y, vx, vy = y
    speed = np.sqrt(vx**2 + vy**2)
    return [vx, vy, -b * vx - c * vx * speed, -m * g - b * vy - c * vy * speed]


m = 1
g = 9.8
b = 0
c = 0
x0 = 0
y0 = 0
vx0 = 4
vy0 = 4
### END SOLUTION2
solution = si.solve_ivp(ode_RHS, (0., 5.), [x0, y0, vx0, vy0],
                        t_eval=np.linspace(0, 5, 2000),
                        args=(m, g, b, c))
x = solution.y[0]  #solution.y is the components of the solution; y[0] is x; y[1] is y, &c.
y = solution.y[1]
plt.plot(x, y)
plt.xlim(0, 3.5)
plt.ylim(0, 1)

(0.0, 1.0)

Now test your solver with drag: set `b=30`, as we did above.

In [11]:
# Test the range against the analytic result here!
known_R = 0.133

#Integrate the ODE using si.solve as above, but set b=30
### BEGIN SOLUTION
solution = si.solve_ivp(ode_RHS, (0.,5.), [0, 0, 4, 4], t_eval=np.linspace(0,5,2000), args=(m, g, 30, 0))
### END SOLUTION

x=solution.y[0]
y=solution.y[1]
print("Range =", x[y>=0][-1])

Range = 0.13333307668201227


In [12]:
# Hidden testing cell; do not delete!

### BEGIN HIDDEN TESTS
solution = si.solve_ivp(ode_RHS, (0.,5.), [0, 0, 4, 4], t_eval=np.linspace(0,5,2000), args=(m, g, 0, 0))
x3=solution.y[0]
y3=solution.y[1]
solution = si.solve_ivp(ode_RHS, (0.,5.), [0, 0, 4, 4], t_eval=np.linspace(0,5,2000), args=(m, g, 30, 0))
x4=solution.y[0]
y4=solution.y[1]

npt.assert_allclose(x3[y3>=0][-1], 3.2616, atol=1e-3)
npt.assert_allclose(x4[y4>=0][-1], 0.1333, atol=1e-3)

npt.assert_allclose(y3[100], 0.6939, atol=1e-3)
npt.assert_allclose(y4[100], 0.06243, atol=1e-3)
### END HIDDEN TESTS

### Part D (15 points)

Now, the grand finale.  You are given a projectile with drag coefficients $b = 0.7$ kg/s and $c = 0.2$ kg/m; the mass is still 1 kg as we've assumed before.  __There are three targets:__ 

* Target 1 is on the ground at $(x,y) = (3,0)$;
* Target 2 is at $(x,y) = (0.5,1)$;
* Target 3 is at $(x,y) = (2,2)$.

Adjusting both the initial speed and angle, there are an _infinite_ number of solutions to this problem!  So let's simplify by saying __the initial launch speed is 10 m/s__.  Assume the targets have __zero width and height__: you must hit the exact coordinates above as closely as possible.  Also, ignore collisions, i.e. you don't have to worry about whether your shot hits a different target than the one you're aiming at.

__Find the launch angles $\theta_1, \theta_2, \theta_3$ (in radians) needed to hit each of the three targets:__ save your answers in the variables `theta_1`, `theta_2` and `theta_3` below.  Your answers will be checked to within _three digits of accuracy after the decimal._

There are no tests for this problem, and no guidelines; all you need to do is hit the targets.  (The only rule is that you must _show your work_ and your code here has to find the angles, i.e. you can't use some other program to solve this and just plug in the numerical values below.)

_(Hint: you should recognize this as a boundary-value problem.  In fact, since we're literally shooting at targets, this seems like an ideal candidate for shooting!)_

_(Another hint: just making plots of where the projectile goes for various choices of the angle is probably a good place to start!)_



__Tricky things you should watch out for:__

1. Now we're evolving in _time_, but our boundary exists in _position_.  So we don't know before starting how many seconds we need to calculate for!  Make sure your `t`-range is long enough that it isn't messing up your shooting solver...

2. Checking the boundary is trickier than for the Schrodinger equation: first we need to find the timestep where our solution for `x` crosses `x_target`, and _then_ we can check how far `y` is from `y_target`.  How can you find the timestep?  Well, this sounds very similar to what I did up in `find_range` above, where we found the timestep where the difference between `y` and 0 was smallest...

3. There are _two_ values of $\theta$ that will hit each of the targets; either solution is fine, although some of them are a little trickier to find and may require a good initial guess.

4. Since there are often two solutions, a bisection root-finder is hard to use here.  The `scipy.optimize.newton` finder worked well for me, but __the default options for `newton` will probably lead to frustration!__  You probably want to adjust both the tolerance option `tol=...` and the maximum number of iterations `maxiter=...`.


(If you manage to find __all six solutions__ for `theta` that will hit the targets, then you will get up to _4 points of bonus credit back_ for any mistakes you made elsewhere on this homework!  You can't do better than a perfect score, though.)

In [13]:
### BEGIN SOLUTION

from scipy.optimize import newton
def solution_shoot_projectile(target, theta_0=np.pi/4, b=0.7, c=0.2, v0=10, tmax=10, points_per_sec=1000):
    """
    Find the launch angle theta that will hit a target,
    given drag coefficients b,c and initial speed v0.
    
    Uses shooting to solve the BVP, like our Schrodinger equation solution
    on tutorial 16.
    """
    
    t = np.linspace(0, tmax, tmax * points_per_sec)
    
    def delta_target(theta):
        v0_theta = (v0 * np.cos(theta), v0 * np.sin(theta))
        xt, yt = solve_motion(t, v0_theta, b=b, c=c)
        
        # Find the closest x to the target...
        i_target = np.abs(xt - target[0]).argmin()
        
        # ...and see how far off the y-coordinate is.
        return yt[i_target] - target[1]
    
    # Now use a root-finder on theta
    theta_opt = newton(delta_target, theta_0, tol=1e-5, maxiter=200)
    xopt, yopt = solve_motion(t, (v0 * np.cos(theta_opt), v0 * np.sin(theta_opt)), b=b, c=c)

    return (xopt, yopt), theta_opt
    
### END SOLUTION

In [14]:
# Save your launch angle solutions in these variables - overwrite them below!
theta_1 = np.pi/4
theta_2 = np.pi/4
theta_3 = np.pi/4


# Draw the targets
plt.figure(figsize=(8,8))
def plot_target(x,y, color):
    plt.plot(x,y, color=color, marker='o', markersize=2)
    plt.plot(x,y, color=color, marker='o', markersize=15, fillstyle='none')
    plt.plot(x,y, color=color, marker='o', markersize=25, fillstyle='none')
    plt.plot(x,y, color=color, marker='o', markersize=35, fillstyle='none')
    
plot_target(3,0, color='red')     # Target 1 - hit this with launch angle theta_1
plot_target(0.5,1, color='green') # Target 2 (--> theta_2)
plot_target(2,2, color='blue')    # Target 3 (--> theta_3)

plt.xlim(0,4)
plt.ylim(0,4)
plt.xlabel('x (m)')
plt.ylabel('y (m)')

# Now plot your solutions to see if they look reasonable!



### BEGIN SOLUTION
(x1, y1), theta_1 = solution_shoot_projectile((3,0))
(x2, y2), theta_2 = solution_shoot_projectile((0.5,1), theta_0=np.pi/3)
(x3, y3), theta_3 = solution_shoot_projectile((2,2), theta_0 = np.pi/3, v0 = 12)

# Alternative shot for target 1
(x4, y4), theta_4 = solution_shoot_projectile((3,0), theta_0=1.05)

# Alternative shot for target 2
(x5, y5), theta_5 = solution_shoot_projectile((0.5,1), theta_0=np.pi/2-0.095)

# Alternative shot for target 3
(x6, y6), theta_6 = solution_shoot_projectile((2,2), theta_0=np.pi/2-0.3, v0 = 10)

plt.plot(x1,y1, color='red')
plt.plot(x2,y2, color='green')
plt.plot(x3,y3, color='blue')
plt.plot(x4,y4, color='red', linestyle='--')
plt.plot(x5,y5, color='green', linestyle='--')
plt.plot(x6,y6, color='blue', linestyle='--')

print(theta_1, theta_2, theta_3, theta_4, theta_5, theta_6)

### END SOLUTION

RuntimeError: Tolerance of -29.484754817371815 reached. Failed to converge after 12 iterations, value is -14.439235915262566.

In [15]:
# Hidden testing cell - do not delete!

# Your answers should be saved in these variables!!
# You will lose credit if they aren't!
print ("theta_1, theta_2, theta_3:")
print (theta_1, theta_2, theta_3)

### BEGIN HIDDEN TESTS
check_1A = np.abs(theta_1 - 0.198479) <= 1e-3
check_1B = np.abs(theta_1 - 1.062109) <= 1e-3
check_2A = np.abs(theta_2 - 1.128313) <= 1e-3
check_2B = np.abs(theta_2 - 1.492687) <= 1e-3
check_3A = np.abs(theta_3 - 0.952090) <= 1e-3
check_3B = np.abs(theta_3 - 1.182853) <= 1e-3

assert check_1A or check_1B
assert check_2A or check_2B
assert check_3A or check_3B

### END HIDDEN TESTS

## theta tests to be added here

theta_1, theta_2, theta_3:
0.31739113834222726 1.1408464461888534 0.9520896279496345


AssertionError: 

-----

### Extra material 

(This stuff is _not_ part of the assignment, just for your information.)

The `find_range` function I gave you above is sort of naive, since it just returns the nearest x-coordinate to the y-value that's nearest zero.  We could imagine improving this by __linear interpolation__: find a pair of (x,y) values bracketing zero, and then draw a line between them.  This makes the function significantly more complicated:

In [16]:
def find_range_interp(x,y):
    # Start with brute-force root finding, NumPy style!
    # Find the timestep i where y is closest to zero.
    # (The particle _starts_ at zero, so throw out the first
    # few points in the path to avoid confusion.)
    i_zero = np.abs(y[5:]).argmin() + 5
    
    # Figure out which way zero is from y[i_zero].
    # Then do a linear interpolation to estimate the zero crossing.
    if y[i_zero - 1] / y[i_zero] < 0:
        # Sign changed to the left of i_zero
        i1 = i_zero - 1
        i2 = i_zero
    elif y[i_zero + 1] / y[i_zero] < 0:
        # Sign changed to the right of i_zero
        i1 = i_zero
        i2 = i_zero + 1
    else:
        # This should never happen if we understand our own algorithm!
        # Make sure the program crashes if we somehow end up in this branch.
        assert False
        
    # If y = mx + b, then y=0 at x=-b/m.
    m = (y[i2] - y[i1]) / (x[i2] - x[i1])
    b = y[i1] - m * x[i1]
    
    return -b/m
    

If you try this above where we used `find_range`, you'll find that it doesn't really help with needing a lot of points to get the range correct!  This is because although we have reduced one source of discretization error with `find_range_interp`, this source of error is _small_ compared to the accumulated error from integrating the entire trajectory using Euler's method!

(If we used a more sophisticated scheme like Runge-Kutta 4-5, then `find_range_interp` might have been more useful to use.)